In [88]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np

train = pd.read_csv('./train.csv', sep=';')
test = pd.read_csv('./test.csv', sep=';')

answer = train.cardio

# age, gender, height, weight, 
# ap_hi, ap_lo, cholesterol, gluc, 
# smoke, alco, active

complete = pd.concat([train, test])
complete = complete.drop(['id', 'cardio'], axis=1)
complete = complete.drop(['active', 'alco', 'smoke'], axis=1)

complete['cholesterol'] = complete.cholesterol.astype('category')
complete['gluc'] = complete.gluc.astype('category')
complete['gender'] = complete.gender.astype('category')
complete = pd.get_dummies(complete)
complete.loc[complete.ap_hi == 0, 'ap_hi'] = complete.ap_hi.mean()
complete.loc[complete.ap_lo == 0, 'ap_lo'] = complete.ap_lo.mean()

complete.loc[complete.ap_hi >= 10000, 'ap_hi'] /= 100
complete.loc[complete.ap_hi >= 300, 'ap_hi'] /= 10
complete.loc[complete.ap_hi < 0, 'ap_hi'] *= -1
complete.loc[complete.ap_hi < 30, 'ap_hi'] *= 10
# complete.loc[complete.ap_hi <= 20, 'ap_hi'] *= 10


complete.loc[complete.ap_lo > 4000, 'ap_lo'] /= 100
complete.loc[complete.ap_lo >= 1000, 'ap_lo'] /= 10
# complete.loc[complete.ap_lo <= 20, 'ap_lo'] *= 10

# complete.loc[complete.ap_hi < complete.ap_lo, 'ap_hi'] += 100

train = complete.head(len(train))
test = complete.tail(len(test))

from xgboost.sklearn import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_predict

new_answer = train.ap_hi
new_train = train.drop('ap_hi', axis=1)

model = XGBRegressor()
print model
predictions = cross_val_predict(model, new_train, new_answer, cv=5)

print r2_score(new_answer, predictions)
print mean_absolute_error(new_answer, predictions)

predicted_ap_hi = predictions

new_answer = train.ap_lo
new_train = train.drop('ap_lo', axis=1)

model = XGBRegressor()
print model
predictions = cross_val_predict(model, new_train, new_answer, cv=5)

print r2_score(new_answer, predictions)
print mean_absolute_error(new_answer, predictions)

predicted_ap_lo = predictions

from lightgbm.sklearn import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, log_loss
from sklearn.model_selection import cross_val_predict, learning_curve

train['predicted_ap_hi'] = predicted_ap_hi
train['predicted_ap_lo'] = predicted_ap_lo

model = LGBMClassifier()
print model
predictions = cross_val_predict(model, train, answer, cv=5)
proba_predictions = cross_val_predict(model, train, answer, cv=5, method='predict_proba')

print log_loss(answer, proba_predictions)
print accuracy_score(answer, predictions)
print classification_report(answer, predictions)

# train_sizes, train_scores, test_scores = learning_curve(model, train, answer, cv=5)

# data = { 'x': train_sizes, 'y': map(lambda x: np.mean(x), train_scores) }
# sns.pointplot(x='x', y='y', data=data)

# data = { 'x': train_sizes, 'y': map(lambda x: np.mean(x), test_scores) }
# sns.pointplot(x='x', y='y', data=data, color='red')

# 0.562083469063
# 7.58538453886

# 0.126931186129
# 4.71456382869

# 0.566973354744
# 0.7337

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
0.561148940314
7.59155047034
XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
0.135509785803
4.70189165296
LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=10, nthread=-1, num_leaves=31,
        obj

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.566975525346
0.733957142857
             precision    recall  f1-score   support

          0       0.71      0.79      0.75     35021
          1       0.76      0.68      0.72     34979

avg / total       0.74      0.73      0.73     70000



In [90]:
pd.options.display.max_rows = 100
train['years'] = (train.age / 365).astype('int')
train['cardio'] = answer
train['ap_hi_diff'] = np.abs(train.ap_hi - train.predicted_ap_hi)
train['ap_lo_diff'] = np.abs(train.ap_lo - train.predicted_ap_lo)
train['orig_ap_hi'] = pd.read_csv('./train.csv', sep=';').ap_hi
train['orig_ap_lo'] = pd.read_csv('./train.csv', sep=';').ap_lo
train.sort_values('ap_hi_diff', ascending=False).drop(['cholesterol_1', 'cholesterol_2', 'cholesterol_3'], axis=1)



/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

,age,ap_hi,ap_lo,height,weight,gender_1,gender_2,gluc_1,gluc_2,gluc_3,predicted_ap_hi,predicted_ap_lo,years,cardio,ap_hi_diff,ap_lo_diff,orig_ap_hi,orig_ap_lo
42334,18716,10.0,108.80000,171,80.0,1,0,1,0,0,157.141708,163.468735,51,1,147.141708,54.668735,1,1088
52851,17770,240.0,20.00000,164,64.0,0,1,1,0,0,121.191307,107.983170,48,1,118.808693,87.983170,24,20
8757,16905,10.0,208.80000,163,63.0,0,1,1,0,0,124.069366,96.158844,46,0,114.069366,112.641156,1,2088
69329,19818,240.0,130.00000,145,62.0,1,0,1,0,0,130.273056,107.399002,54,1,109.726944,22.600998,240,130
40330,20322,30.9,96.34799,162,50.0,1,0,1,0,0,140.207840,93.846397,55,0,109.307840,2.501593,309,0
67710,19687,240.0,140.00000,168,75.0,0,1,1,0,0,132.272507,104.377502,53,1,107.727493,35.622498,240,140
9557,22659,70.1,110.00000,155,87.0,1,0,1,0,0,165.819138,85.298660,62,1,95.719138,24.701340,701,110
29985,21331,80.0,120.00000,163,102.0,1,0,1,0,0,175.436981,110.349297,58,1,95.436981,9.650703,80,120
49523,20330,200.0,140.00000,162,67.0,1,0,0,0,1,105.846275,106.827217,55,1,94.153725,33.172783,200,140
20697,22626,70.0,110.00000,165,61.0,1,0,0,0,1,163.533340,80.718376,61,0,93.533340,29.281624,70,110


In [57]:
train['predicted'] = predictions
train['expected'] = answer
train['probas'] = proba_predictions[:, 1]
train['loss'] = - ((answer * np.log(proba_predictions[:, 1]) + (1 - answer) * np.log(1 - proba_predictions[:, 1])) / len(answer))
errors = train[train['predicted'] != train['expected']]
errors.sort_values('loss', ascending=False)
# sns.pairplot(errors[['ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'age', 'weight', 'gender', 'height', 'body_mass_index', 'expected', 'predicted', 'probas']], markers='+', hue='expected', plot_kws={'s': 1}, size=4)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [92]:
train[train.ap_hi == 10]

,age,ap_hi,ap_lo,height,weight,cholesterol_1,cholesterol_2,cholesterol_3,gender_1,gender_2,...,gluc_2,gluc_3,predicted_ap_hi,predicted_ap_lo,years,cardio,ap_hi_diff,ap_lo_diff,orig_ap_hi,orig_ap_lo
8757,16905,10.0,208.8,163,63.0,1,0,0,0,1,...,0,0,124.069366,96.158844,46,0,114.069366,112.641156,1,2088
42334,18716,10.0,108.8,171,80.0,1,0,0,1,0,...,0,0,157.141708,163.468735,51,1,147.141708,54.668735,1,1088


In [208]:
import time

model.fit(train, answer)
proba_predictions = model.predict_proba(test)

submission_response = pd.DataFrame({
    'y': proba_predictions[:,1]
})

timestr = time.strftime("%Y%m%d_%H%M")
pd.DataFrame(submission_response).to_csv('./outputs/naive.' + timestr + '.csv', header=False, index=False, line_terminator="\n")
pd.DataFrame(submission_response).to_csv('./outputs/_naive.latest.csv', header=False, index=False, line_terminator="\n")

In [64]:
pd.options.display.max_rows = 1000
complete.ap_lo.sort_values()

24125      -90.0
60106      -70.0
35140        1.0
16009        6.0
28065        6.0
68568        6.0
26877        7.0
18898        7.0
6870         7.0
19075        7.0
7598         8.0
17093        8.0
20074        8.0
9777         8.0
27148        9.0
38599        9.0
6737        10.0
26513       10.0
37169       10.0
2443        10.0
6400        10.0
10106       10.0
63828       10.0
47711       10.0
7638        10.0
43307       10.0
11068       10.0
8295        12.0
56927       15.0
17406       19.0
24757       20.0
29836       20.0
11820       20.0
68223       20.0
52851       20.0
40650       20.0
29578       20.0
39896       20.0
8797        20.0
26653       20.0
24003       20.0
20135       20.0
13935       20.0
44609       20.0
6294        20.0
18958       20.0
57641       20.0
23305       20.0
40961       20.0
22380       20.0
36958       30.0
5333        30.0
12852       30.0
50446       30.0
33262       30.0
418         30.0
1626        30.0
2371        30.0
62031       30